
### Roll : 21CS60A02
### Name : Md Ashik Khan

### Machine Learning
### Project 1: Decision Trees

In [29]:
import pandas as pd
import numpy as np
from math import log
from pprint import pprint

In [30]:
train = pd.read_csv("project1 1.data",header=None, names=['price','maint','doors','persons','lug_boot','safety','decision'])
test = pd.read_csv("project1_test.data",header=None, names=['price','maint','doors','persons','lug_boot','safety','decision'])
train.head()

price  maint  doors  persons lug_boot safety decision
0  vhigh  vhigh      2        2    small    low    unacc
1  vhigh  vhigh      2        2    small    med    unacc
2  vhigh  vhigh      2        2    small   high    unacc
3  vhigh  vhigh      2        2      med    low    unacc
4  vhigh  vhigh      2        2      med    med    unacc

# Classification 

In [31]:
def classification(decisionTree, testData):
    feature = list(decisionTree.keys())[0]
    try:
        branch = decisionTree[feature][testData[feature]]
    except:
        branch = decisionTree[feature][list(decisionTree[feature].keys())[0]]
    if isinstance(branch, dict):
        return classification(branch, testData)
    else:
        return branch

# Information Gain based Decision Tree Model

In [32]:
###defining entropy
def calc_entropy(inptArr):
    #get individual values
    uniqueClass, countClass = np.unique(inptArr[:, -1], return_counts = True)
    entropy = 0
    i = 0
    while i < len(uniqueClass):
         #formula
        entropy += -(countClass[i]/len(inptArr))*log((countClass[i]/len(inptArr)), 2)
        i += 1
    return entropy

In [33]:
##Info Gain
def calc_infoGain(inptdf, inptArr, attribute):
    #calculate total entropy
    entropyS = calc_entropy(inptArr)
    attrIndex = np.where(np.array(inptdf.columns) == attribute)[0][0]
    attrVal, valCount = np.unique(inptArr[:, attrIndex], return_counts = True)
    infoGain = 0
    entSum = 0
    i = 0
    for i in range(0, len(attrVal)):
        value_frac = valCount[i]/len(inptArr)
        curr = inptArr[inptArr[:, attrIndex] == attrVal[i]]
        #cal the weighted entropy
        entSum += value_frac * calc_entropy(curr)
    #formula for information gain
    infoGain = entropyS - entSum
    return infoGain

In [34]:
def best_attribute(inptdf, inptArr):
    attrSet = np.array(inptdf.columns)
    maxInfoGain = -1
    bestAttr = 'NULL'
    for i in range(0, (len(attrSet) - 1)):
        infoGain = calc_infoGain(inptdf, inptArr, attrSet[i])
        if infoGain > maxInfoGain:
            maxInfoGain = infoGain
            bestAttr = attrSet[i]
    return bestAttr

In [35]:
def val_dataframe(inptdf, attr,value):
    subT = inptdf[inptdf[attr] == value]
    pd.RangeIndex(len(subT.index))
    return subT

In [36]:
def decision_tree(inptdf, decisionTree = None):
    inptArr = inptdf.values
    attr = best_attribute(inptdf, inptArr)
    attrIndex = np.where(np.array(inptdf.columns) == attr)[0][0]
    attrVal = np.unique(inptArr[:, attrIndex])
    #if the dataset is empty
    if decisionTree is None:
        decisionTree = {}
        decisionTree[attr] = {}
    #Grow the tree branch under the root node
    i = 0
    while i < len(attrVal):
        subT = val_dataframe(inptdf, attr, attrVal[i])
        classVal = np.unique(subT[subT.columns[-1]])
        if len(classVal) != 1:
            decisionTree[attr][attrVal[i]] = decision_tree(subT)
        else:
            decisionTree[attr][attrVal[i]] = classVal[0]
        i += 1
    return decisionTree

# Accuracy function

In [37]:
def prediction(decisionTree, test):
    tree_true = []
    tree_pred = []
    for i in range(len(test)):
        #classification
        result = classification(decisionTree, test.iloc[i])
        tree_true.append(test.iloc[i][-1])
        tree_pred.append(result)
    
    return tree_true,tree_pred

In [38]:
def correct_error_and_accuracy(decisionTree, test):
    matches = []
    tree_true = []
    tree_pred = []
    for i in range(len(test)):
        #prediction result
        result = classification(decisionTree, test.iloc[i])
        matches.append((result, test.iloc[i][-1]))
        #testing data
        tree_true.append(test.iloc[i][-1])
        #prediction data
        tree_pred.append(result)

    
    count_correct = 0
    count_error = 0
    for tup in matches:
        if tup[0] == tup[1]:
            count_correct += 1
        else:
            count_error += 1
    #gaining accuracy
    accuracy = count_correct/len(matches)*100
    return count_correct, count_error,accuracy

# Result

In [39]:
DTree = decision_tree(train)

In [40]:
pprint(DTree)

{'safety': {'high': {'persons': {2: 'unacc',
                                 4: {'price': {'high': {'maint': {'high': 'acc',
                                                                  'low': 'acc',
                                                                  'med': 'acc',
                                                                  'vhigh': 'unacc'}},
                                               'low': {'maint': {'high': {'lug_boot': {'big': 'vgood',
                                                                                       'med': {'doors': {2: 'acc',
                                                                                                         3: 'acc',
                                                                                                         4: 'vgood',
                                                                                                         6: 'vgood'}},
                                                           

In [41]:
pred,label = prediction(DTree,test)
print("Predictions: ",pred)
print("Labels:",label)


Predictions:  ['unacc', 'unacc', 'acc', 'acc', 'unacc', 'acc', 'acc', 'good', 'unacc', 'good', 'vgood', 'good', 'unacc', 'good', 'vgood', 'acc', 'vgood']
Labels: ['unacc', 'unacc', 'acc', 'acc', 'unacc', 'acc', 'acc', 'good', 'unacc', 'good', 'vgood', 'good', 'unacc', 'good', 'vgood', 'acc', 'vgood']


In [42]:
count_correct, count_error,accuracy = correct_error_and_accuracy(DTree, test)
print('Correct predictions of Decision Tree : '+str(count_correct))
print('Errors of Decision Tree : '+str(count_error))
print('Accuracy of Decision Tree : '+str(accuracy)+'%')

Correct predictions of Decision Tree : 17
Errors of Decision Tree : 0
Accuracy of Decision Tree : 100.0%


<h1><center>END</center></h1>